In [13]:
# IMPLEMENTATION OF DISPERSIVE MEDIA USING THE DRUDE MODEL

import numpy as np
from matplotlib import pyplot as plt
from matplotlib.backends.backend_pdf import PdfPages

# defining the simulation volume
nm = 1e-9
size = 10000*nm
dz = 10*nm
Nz = (int) (size//dz)
Nt = 5000

# constants
# epsilon_r = 1 #vacuum
epsilon_0 = 8.85e-12 #free space permittivity
c0 = 3e8
sigma = 0 #non-conductive media being considered

# time step via Courant condition
dt = dz/(2*c0) 

# initialising E and B throughout the space grid as 0
E = np.zeros(Nz)
B = np.zeros(Nz)
P = np.zeros(Nz)
P_old = np.zeros(Nz) # one timestep back
P_older = np.zeros(Nz) # two timesteps back

Lambda = 800*nm #Ti-Sa laser being considered
omega = 2*np.pi*c0/Lambda # frequency

n0 = 0 # density of material
q = 1.6e-19 # electron charge (C)
m_e = 9.1e-31 # electron mass (kg)
# omega_plasma = np.sqrt(n0*q**2/(epsilon_0*m_e))
omega_plasma = 1.3e16 # rad/s
gamma = 2.7e13 # damping factor/collision frequency

iota = 1j

# epsilon_r for Si
epsilon_r = 1 - omega_plasma**2/(omega** + iota*omega*gamma)



In [14]:
source_pos = Nz//2
E0 = 1
#inject sinusoidal source in the middle of the grid

with PdfPages("#2a_DielectricSlab.pdf") as pdf:
    
    E_left =0
    B_right=0
    for n in range(0, Nt): # 1:Nmax time steps
        
        # update the old polarisation arrays
        P_older = P_old
        P_old = P
        for k in range(1, Nz) :
            # general update equations for the current polarisation array P
            const1 = (1/(2*dt)**2 - gamma)
            const2 = (1/(2*dt)**2 + gamma)
            
            P[k] = (epsilon_0*omega_plasma**2*E[k] + (1/(2*dt)**2)*P_old[k] - const1*P_older[k])
        
        
        # UPDATE FOR E PART
        temp = E[1] # for the left side of the grid
        for k in range (1,Nz): # 2:last
            E[k] = E[k] - (c0*dt/dz)*(B[k]-B[k-1]) - (1/epsilon_0)(P[k] - P_old[k])  
            
        E[source_pos] += E0*np.sin(omega*n*dt)
        
        E[0] = E_left
        E_left = temp
        
            
        temp = B[-2] # for the right side of the grid
        for k in range(0,Nz-1): # 1:last-1
            B[k] = B[k] - dt/dz*(E[k+1] - E[k])
        
        B[-1] = B_right
        B_right = temp
        
        #exporting the plots to a pdf
        if(n%20 == 0) :
            plt.figure(figsize=(8, 6))
            plt.plot(E, "-r", label="E-field")
            plt.xlabel('z')
            plt.ylabel('E(z)')
            plt.legend(loc="upper left")
            plt.title(f'At the timestep {n}')
            plt.suptitle("Sinusoidal source, Silver (Dispersive Media)")
            plt.ylim(-4,4)
            plt.grid(True)
            
            # highlighting the source and the slab
            plt.axvline(x = source_pos, color= "r")
            plt.axvspan(0, 1000, color='k', alpha=0.05)
            
            pdf.savefig()
            plt.close()

TypeError: 'float' object is not callable